In [23]:
import pickle
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

In [24]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

In [25]:
(x_train, t_train), (x_test, t_test) = \
    load_mnist(flatten=True, normalize=False) #normalize 는 입력 이미지의 픽셀값을 0.0 ~ 1.0 사이의 값으로 정규화 할지 정함 / False니깐 0~255 를 유지하기
#flatten은 입력 이미지를 평탄하게 즉 1차원 배열로 만들지 결정하는데 True 이기 때문에 1차원 배열로(784개의 원소) / False 이였으면 1 X 28 X 28 인 3차원 배열로 생성

In [26]:
#데이터 형상 출력하기

print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [27]:
img = x_train[0]
label = t_train[0]
print(label)

5


In [28]:
print(img.shape)
img = img.reshape(28, 28) #원래 이미지 모양으로 변형
print(img.shape)

print(img_show(img))

(784,)
(28, 28)
None


In [29]:
#추론처리

def get_data():
    (x_train, t_train) , (x_test, t_test) = \
        load_mnist(normalize=True, flatten=True, one_hot_label=False) #one_hot_label 은 레이블을 형태로 저장할지 결정함 ex): [0,0,0,1,0,0,0] 처럼 정답을 뜻하는건 1(hot)이고 나머지 0은 one 0인 배열
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl" , 'rb') as f:
        network = pickle.load(f)

    return network


def sigmoid(x): # 시그모이드 구현
    return 1 / (1 + np.exp(-x))

def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y

def predict(network, x):
    W1 , W2 , W3 = network['W1'] , network['W2'] , network['W3']
    b1 , b2 , b3 = network['b1'] , network['b2'] , network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

In [30]:
#정확도

x, t = get_data()
network = init_network()

accuracy_cnt = 0

for i in range(len(x)): # x 길이 만큼 반복하기
    y = predict(network, x[i])
    p = np.argmax(y)
    if p == t[i]:
        accuracy_cnt += 1

print("정확도:" + str(float(accuracy_cnt) / len(x)))

정확도:0.098


In [31]:
#배치 처리

x , _ = get_data()
network = init_network()
W1 , W2 , W3 = network['W1'] , network['W2'] , network['W3']

print(x.shape)
print(x[0].shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)

(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


In [32]:
x , t = get_data()
network = init_network()

batch_size = 100 # 배치크기
accuracy_cnt = 0

for i in range(0, len(x) , batch_size):
    x_batch = x[i:i + batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis = 0)
    accuracy_cnt += np.sum(p == t[i:i + batch_size])

print("정확도 :" + str(float(accuracy_cnt) / len(x)))

정확도 :0.098
